In [9]:
# Import numpy and matplotlib
%load_ext line_profiler
from collections import namedtuple
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import sys
import time
sys.path.append('../scripts/')
from eulerian_functions import *

# This code forces re-import of modules every time a cell is executed.
# Useful for development.
%load_ext autoreload
%autoreload 2

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
########################################
#### Scenario parameters for Case 1 ####
########################################

#### Hard-coded parameters for this case ####
# Total depth
Zmax = 50
# Simulation time
Tmax = 24*3600

# For this case, we use a speed distribution directly, taken from
# Table 3 in Sundby (1983).
# Mean speed = 0.96 mm/s
# Standard deviation = 0.38 mm/s
# Truncated at +/- 2*sigma
mean_speed = 0.96 * 1e-3
std_dev_speed = 0.38 * 1e-3
Vmin = mean_speed - 2*std_dev_speed
Vmax = mean_speed + 2*std_dev_speed
speed_distribution = lambda v: np.exp(-0.5*((v - mean_speed)/std_dev_speed)**2) / (std_dev_speed*np.sqrt(2*np.pi))

# Initial condition:
# Normal distribution with mean mu and standard deviation sigma
sigma_IC = 4
mu_IC = 20
pdf_IC = lambda z: np.exp(-0.5*((z - mu_IC)/sigma_IC)**2) / (sigma_IC*np.sqrt(2*np.pi))


##################################
####   Diffusivity profiles   ####
##################################

# Constant diffusivity
K_A = lambda z: 1e-2*np.ones(len(z))

# Fitted to results of GOTM simulation
alpha, beta, zeta, z0 = (0.00636, 0.088, 1.54, 1.3)
K_B = lambda z: alpha*(z+z0)*np.exp(-(beta*(z+z0))**zeta)


####################################################
####   Populate object with system parameters   ####
####################################################

params = EulerianSystemParameters(
        Zmax = Zmax, # Max depth of water column
        Nz = 2000, # Number of cells in z-direction
        Tmax = Tmax, # Simulation time
        dt = 3600, # timestep
        Vmin = Vmin, # Minimum speed
        Vmax = Vmax, # maximum speed
        Nclasses = 16, # Number of speed classes
        speed_distribution = speed_distribution, # speed density
    )


###########################################################
####    Run simulation for both diffusivity profiles   ####
###########################################################


# Initial concentration array for all cells and time levels
C0 = pdf_IC(params.z_cell)[None,:] * params.mass_fractions[:,None]
#C0 = pdf_IC(params.z_cell)[:,None] * params.mass_fractions[None,:]

tic = time.time()
c = Crank_Nicolson_FVM_TVD_advection_diffusion_reaction(C0, K_B, params)
toc = time.time()

print(f'Simulation took {toc - tic:.4f} seconds')

100%|██████████| 24/24 [00:04<00:00,  5.22it/s]

Simulation took 4.7982 seconds


In [14]:
%lprun -f Iterative_Solver c = Crank_Nicolson_FVM_TVD_advection_diffusion_reaction(C0, K_B, params)

100%|██████████| 24/24 [00:04<00:00,  4.92it/s]
